<a href="https://colab.research.google.com/github/yleessam/tf/blob/main/transfer/transfer_learning_%EA%B2%AC%EC%A2%85%EC%9D%B8%EC%8B%9D%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 라이브러리 import

In [1]:
#import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# Stanford Dogs 데이터셋 다운로드 및 압축 해제

In [2]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!tar -xf images.tar

--2024-02-05 19:06:35--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  10.9MB/s    in 77s     

2024-02-05 19:07:53 (9.80 MB/s) - ‘images.tar’ saved [793579520/793579520]



# 데이터 준비

- Stanford Dogs 데이터셋을 학습 및 검증 데이터셋으로 나누어 줍니다.
- Keras의 ImageDataGenerator를 사용하여 데이터를 로드하고 전처리합니다.


In [3]:
#훈련 데이터 증강

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2) # 검증 데이터를 위한 분할 비율


In [4]:
#훈련 데이터 생성
train_generator = train_datagen.flow_from_directory(
    'Images', # 다운로드 및 압축 해제한 폴더
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training') # 학습 데이터

Found 16508 images belonging to 120 classes.


In [5]:
#검증 데이터 생성
validation_generator = train_datagen.flow_from_directory(
    'Images', # 같은 폴더에서 검증 데이터 생성
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # 검증 데이터


Found 4072 images belonging to 120 classes.


# 모델구성

In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 1s 0us/step


In [7]:

# 기본 모델의 파라미터를 동결
for layer in base_model.layers:
    layer.trainable = False



1. GlobalAveragePooling2D()는 컨볼루션 신경망(CNN)에서 사용되는 레이어 중 하나로, 특성 맵(feature map)의 글로벌 평균 값을 계산합니다. 이 레이어는 2차원 입력 맵의 각 채널에 대해 평균 값을 계산하여, 각 채널마다 단 하나의 숫자를 출력합니다. 결과적으로, 이 레이어는 2D 특성 맵을 채널별 평균값으로 축소하여 1D 벡터로 변환합니다.

> 예를 들어, GlobalAveragePooling2D() 레이어가 입력으로 [height, width, channels] 형태의 특성 맵을 받으면, 각 채널의 모든 height x width 픽셀 값들의 평균을 계산하여 [channels] 형태의 벡터를 출력합니다. 이 과정은 각 채널의 공간적 정보를 단일 대표값으로 압축하여, 모델이 공간적 위치보다는 채널별 통계적 정보에 더 집중하도록 합니다.




2. GlobalAveragePooling2D()의 주요 장점 중 하나는 모델의 파라미터 수를 크게 줄여주며, 이는 과적합(overfitting)을 줄이는 데 도움이 될 수 있습니다. 또한, 이 레이어는 입력 특성 맵의 크기에 무관하게 동작하기 때문에, 다양한 크기의 입력 이미지를 처리할 수 있는 모델을 만들 때 유용합니다.

3. 이 레이어는 특히 전역적인 컨텍스트 정보를 요약하기 위해 사용되며, 분류 작업에서 마지막 컨볼루션 레이어 뒤에 배치되어 특성을 하나의 벡터로 압축하는 데 자주 사용됩니다.

In [9]:

# 커스텀 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) # 새로운 FC 레이어
predictions = Dense(train_generator.num_classes, activation='softmax')(x) # 최종 레이어

In [10]:
# 새로운 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)


# 모델 컴파일

In [11]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# 모델 학습

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

Epoch 1/10
 28/515 [>.............................] - ETA: 46:46 - loss: 4.4440 - accuracy: 0.1406

# 모델 평가

In [ ]:
# 모델 평가
eval_result = model.evaluate(validation_generator)
print(f'Validation loss: {eval_result[0]}, Validation accuracy: {eval_result[1]}')
